## `variable_scope`: Basic idea

- Variable Scope: with two major api
  - `tf.variable_scope(<scope_name>)`: manage the variables got from `tf.get_variable`
  - `tf.get_variable(<name>, <shape>, <initializer>)`: Creates or returns a variable with a given name.
    - `tf.constant_initializer(value)`
    - `tf.random_uniform_initializer(a, b)`
    - `tf.random_normal_initializer(mean, stddev)`
    - an initial value
    - or any function that takes the shape and returns a tensor with that shape
- `tf.get_variable_scope` will get the current variable scope
- variable scope add prefixes to the name of tensors you created within that scope
  - `tf.variable_scope` creates a variable scope
  - `tf.get_variable` is the major api to get and reuse tensors within a variable scope

Notes:
- **tf_variable_op_scope** has been deprecated.

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf

In [2]:
def my_initializer(shape, dtype=None, partition_info=None):
    init_value = tf.zeros(shape, dtype)
    init_value = 3690
    return init_value

In [3]:
# Simple example
with tf.variable_scope("my_scope"):
    x = tf.constant(3, name="x")
    # if it is the first time you create variable with tf.get_variable,
    # you have to specify the sahpe explicitly.
    y = tf.get_variable(name="y", shape=[0], initializer=my_initializer)
    z = tf.get_variable(name="z", initializer=30.0)

In [4]:
print(x.name)
print(y.name)
print(z.name)

my_scope/x:0
my_scope/y:0
my_scope/z:0


`<scope>/<op_name>:<output_index>`

In [5]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print("x: {}".format(sess.run(x)))
    print("y: {}".format(sess.run(y)))
    print("z: {}".format(sess.run(z)))

x: 3
y: 3690.0
z: 30.0


In [6]:
# How to share vairalbe? "reuse" it
try:
    with tf.variable_scope("my_scope"):
        y2 = tf.get_variable(name="y")
except ValueError as e:
    print(e)
    print("You need to reuse variable!")

Variable my_scope/y already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-3-cd72fe1b9ecb>", line 6, in <module>
    y = tf.get_variable(name="y", shape=[0], initializer=my_initializer)
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2802, in run_ast_nodes
    if self.run_code(code, result):

You need to reuse variable!


In [7]:
with tf.variable_scope("my_scope", reuse=True):
    y3 = tf.get_variable(name="y")
    if y3 is y:
        print("I get the same y!")
        
    z2 = tf.get_variable("z")
    if z2 is z:
        print("I get the same z")

I get the same y!
I get the same z


In [8]:
# or you can use scope.resue_variables() to reuse variables
with tf.variable_scope("my_scope") as scope:
    scope.reuse_variables()
    y3_2 = tf.get_variable(name="y")
    if y3_2.name == y.name:
        print("Get the same y!")

Get the same y!


In [9]:
try:
    with tf.variable_scope("my_scope", reuse=True):
        q = tf.get_variable(name="q", 
                            shape=[3, 3], 
                            initializer=tf.random_normal_initializer(0.0, 1.0))
    with tf.Session() as sess:
        print(sess.run(q))
except Exception as e:
    print(e)
    print()
    print("You use a variable which hasn't been created before.")
    print("A `ValueError` will be throwed!")

Variable my_scope/q does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=None in VarScope?

You use a variable which hasn't been created before.
A `ValueError` will be throwed!


In [10]:
try:
    with tf.variable_scope("my_scope", reuse=True):
        yy = tf.get_variable(name="y", shape=[1])
except ValueError as e:
    print(e)
    print()
    print("The name and the shape must be the same as the time when the variable is created")

Trying to share variable my_scope/y, but specified shape (1,) and found shape ().

The name and the shape must be the same as the time when the variable is created


In [11]:
print(tf.get_variable_scope().name is '') # default scope, ''
with tf.variable_scope("hello_scope"):
    print(tf.get_variable_scope().name)

True
hello_scope


### Understanding `tf.get_variable`

Two scenerio:

1. the scope is set for creating **new** variables
  - That is, `tf.get_variable_scope().reuse == False`
  - If the variable you want to created has been created before (the same name under the same variable scope), this will results in a `ValueError`.
2. the scope is set for sharing variables
  - That is, `tf.get_variable_scope().reuse == True`
  - In this case, `tf.get_variable` will search for existing variable by its name (required) and its shape (optional).
    - `tf.get_variable` will check the shape if given with the variable it find with name

In [12]:
# This example is adopted from the document
with tf.variable_scope("foo") as foo_scope:
    assert foo_scope.name == "foo"
with tf.variable_scope("bar"):
    with tf.variable_scope("baz") as other_scope:
        assert other_scope.name == "bar/baz"
        with tf.variable_scope(foo_scope) as foo_scope2:
            assert foo_scope2.name == "foo"  # Not changed.

Opening a variable scope using a previously existing scope, we will jump out of the current variable scope prefix to an entirely different one. 

That is, it's fully independent of where we do it.

In [13]:
# You can also use nested scope
with tf.variable_scope("scope1"):
    y = tf.get_variable(name="y", initializer=3.0)
    with tf.variable_scope("scope2"):
        z = tf.get_variable(name="z", shape=[3], initializer=tf.constant_initializer(3.0))

print(y.name)
print(z.name)

scope1/y:0
scope1/scope2/z:0


In [14]:
# variable scope will effect the name of ops, too
with tf.variable_scope("my_ops"):
    a = tf.get_variable(name='a', initializer=3.0)
    b = tf.get_variable(name='b', initializer=5.0)
    x = a + b
print(x.name)

my_ops/add:0


### Variable Scope Summary:

- Using variable scope with `tf.variable_scope` and `tf.get_variable`
- Explicifically set `reuse=True` or calling `scope.reuse_variables` to reuse variables
  - If you use a variable that is not created yet in a reuse variable scope, a `ValueError` exception will be throwed.
  - The mechenism prevent you from accidentally use shared variable.
- Variable scope add name prefixes to both variables and operations
- `variable_scope` is mainly for sharing variables

-----

## `tf.name_scope`: The Basic Idea

Well, the purpose of `tf.variable_scope` is kinda clear now. But what is `tf.name_scope` for anyway?

According to the [doc](https://www.tensorflow.org/api_docs/python/tf/name_scope), it will return a contex manager for use in defining ops. Along with the [doc](https://www.tensorflow.org/api_docs/python/tf/Graph#name_scope) for `tf.Graph.name_scope`, `name_scope` is for wrapping tensors and ops into one conceptually unit.

Let's see some examples.

In [15]:
with tf.name_scope(name="my_name"):
    x = tf.Variable(np.random.randn(3, 5),
                    name="x",
                    dtype=tf.float32)
    y = tf.get_variable(name="y",
                        initializer=5.0)
    output = x + y

In [16]:
print(x.name)
print(y.name)
print(output.name)

my_name/x:0
y:0
my_name/add:0


As we can see in this example, that `tf.name_scope` will

1. add prefix to the name of tensors created through `tf.Variable` and operations.
2. It has no effect to variable created by `tf.get_variable`.

Up to this point, we can say that `tf.name_scope` is just another way to manage scope of variables and ops in `Tensorflow`

```
tf.name_scope(
    name,
    default_name=None,
    values=None
)
```

According to the [doc](https://www.tensorflow.org/api_docs/python/tf/name_scope), `tf.name_scope` create a name scope and do following things:

1. check the given `values` are from the same graph
2. making that graph as default graph
3. push the name scope created to the graph

In [17]:
# example from the doc
def relu_layer(x, out_shape, name=None):
    if not isinstance(x, tf.Tensor):
        x = tf.convert_to_tensor(x, dtype=tf.float32)
    with tf.name_scope(name, "MyRelu", [x]) as scope:
        # scope name will be "MyRelu" if `name` is None.
        weight = tf.Variable(tf.random_normal([x.shape[1].value, out_shape]),
                             name="weight",
                             dtype=tf.float32)
        bias = tf.Variable(tf.random_normal([out_shape]),
                           name="bias",
                           dtype=tf.float32)
        affine = tf.matmul(x, weight) + bias
        output = tf.nn.relu(affine, name=scope)
    return output

In [19]:
x = tf.placeholder(tf.float32, shape=[None, 10])
layer1 = relu_layer(x, 30, name="relu")
print(layer1.name)

relu:0


**Note**: `name_scope` also support nested scoping, see example from [doc](https://www.tensorflow.org/api_docs/python/tf/Graph#name_scope) for detail 

----

## When and Why

So when to use `variable_scope` and when to use `name_scope`?

IMHO, I summarize the following principles:

1. If you are about to define variables which can be used in other operation in the graph, go for the `tf.variable_scope` and `tf.get_variable`. They are all about sharing variables.
2. If you are about to wrap variables into one operation, go for `tf.name_scope`. It's about wrapping tensors and ops into one named scope. You have to be aware of the fact that `tf.get_variable` will not be governed by `name_scope`.

## References

- [StackOverflow](https://stackoverflow.com/questions/35919020/whats-the-difference-of-name-scope-and-a-variable-scope-in-tensorflow)
- [StackOverflow](https://stackoverflow.com/questions/34215746/difference-between-variable-scope-and-name-scope-in-tensorflow)
- [StackOverflow - 'meaning of :0'](https://stackoverflow.com/questions/40925652/in-tensorflow-whats-the-meaning-of-0-in-a-variables-name)
- [StackOverflow - 'Tensor naming'](https://stackoverflow.com/questions/36150834/how-does-tensorflow-name-tensors)
- [jasdeep06.github.io](https://jasdeep06.github.io/posts/variable-sharing-in-tensorflow/)
- [doc](https://www.tensorflow.org/programmers_guide/variable_scope)